In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import pickle
from sklearn import preprocessing

In [2]:
pd.set_option('display.max_columns', 500)  # or 1000

In [3]:
datadir = '~/Downloads/datasets-interactive-top-tuples/'
fname = 'games'
fdata = fname + '.csv'

In [73]:
df = pd.read_csv(datadir+fdata, delimiter=',') # na_values='NaN'

In [74]:
df = df.drop(['AppID','Name','Release date','About the game','Full audio languages',
              'Reviews','Header image','Website', 'Support url', 'Support email', 
              'Metacritic url','Score rank','Notes','Developers','Publishers','Tags', 'Screenshots', 'Movies',
              # 'Required age','Windows','Mac', 'Linux', 'Supported languages',
             ],
        axis=1)

In [75]:
df

,Estimated owners,Peak CCU,Required age,Price,DLC count,Supported languages,Windows,Mac,Linux,Metacritic score,User score,Positive,Negative,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Categories,Genres
0,0 - 20000,0,0,19.99,0,['English'],True,False,False,0,0,6,11,30,0,0,0,0,0,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports"
1,0 - 20000,0,0,0.99,0,"['English', 'French', 'Italian', 'German', 'Sp...",True,True,False,0,0,53,5,12,0,0,0,0,0,"Single-player,Steam Achievements,Full controll...","Action,Indie"
2,0 - 20000,0,0,4.99,0,"['English', 'Portuguese - Brazil']",True,False,False,0,0,0,0,0,0,0,0,0,0,Single-player,"Action,Adventure,Indie,Strategy"
3,0 - 20000,0,0,5.99,0,"['English', 'French', 'Italian', 'German', 'Sp...",True,True,True,0,0,3,0,0,0,0,0,0,0,"Single-player,Full controller support","Adventure,Casual,Indie"
4,0 - 20000,0,0,0.00,0,"['English', 'Spanish - Spain']",True,True,False,0,0,50,8,17,0,0,0,0,0,"Single-player,Steam Achievements","Adventure,Indie"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63205,0 - 0,0,0,0.00,2,['English'],True,False,False,0,0,0,0,0,0,0,0,0,0,"Multi-player,PvP,Online PvP","Action,Early Access"
63206,0 - 20000,3,0,9.99,0,['English'],True,False,False,0,0,2,0,15,0,0,0,0,0,"Single-player,Steam Achievements,Full controll...","Action,Adventure,Indie"
63207,0 - 0,0,0,7.19,0,['Simplified Chinese'],True,False,False,0,0,0,0,9,911,0,0,0,0,Single-player,"Indie,Early Access"
63208,0 - 0,0,0,2.99,0,"['English', 'Russian', 'Simplified Chinese']",True,False,False,0,0,0,0,0,0,0,0,0,0,"Single-player,Partial Controller Support,Remot...","Casual,Indie,Racing,Simulation,Sports"


In [76]:
df.isna().sum()

Estimated owners                 0
Peak CCU                         0
Required age                     0
Price                            0
DLC count                        0
Supported languages              0
Windows                          0
Mac                              0
Linux                            0
Metacritic score                 0
User score                       0
Positive                         0
Negative                         0
Achievements                     0
Recommendations                  0
Average playtime forever         0
Average playtime two weeks       0
Median playtime forever          0
Median playtime two weeks        0
Categories                    2646
Genres                        1738
dtype: int64

In [77]:
df = df.dropna()
df.shape

(60496, 21)

In [78]:
df1 = df['Supported languages'].str.replace('[','').str.replace(']','').str.replace("'",'').str.replace(' ','').str.get_dummies(sep = ',')

<ipython-input-78-440221847316>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df1 = df['Supported languages'].str.replace('[','').str.replace(']','').str.replace("'",'').str.replace(' ','').str.get_dummies(sep = ',')


In [79]:
# df1.columns.values.tolist()
df1 = df1[['English','French','Japanese','Swedish','Italian','German','Arabic',
           'Finnish','Greek','Hebrew','Persian','Russian','Korean']]
df = pd.concat([df, df1], axis=1)
df = df.drop(['Supported languages'], axis=1)
df.shape

(60496, 33)

In [80]:
df['Estimated owners'].unique().shape, df['Categories'].unique().shape, df['Genres'].unique().shape

((14,), (5014,), (1943,))

In [81]:
df['Estimated owners'].unique()

array(['0 - 20000', '50000 - 100000', '20000 - 50000', '200000 - 500000',
       '100000 - 200000', '2000000 - 5000000', '0 - 0',
       '500000 - 1000000', '1000000 - 2000000', '20000000 - 50000000',
       '5000000 - 10000000', '10000000 - 20000000',
       '50000000 - 100000000', '100000000 - 200000000'], dtype=object)

In [82]:
df['Estimated owners'] = df['Estimated owners'].str.split(' - ').str[0].astype(int)

In [83]:
for c in ['Categories','Genres']:
    df1 = df[c].str.get_dummies(sep = ',')
    df = pd.concat([df, df1], axis=1)

df = df.drop(['Categories','Genres'], axis=1)
df.shape

(60496, 100)

In [84]:
for c in ['Windows','Mac', 'Linux']:
    df[c] = df[c].astype(int)

In [85]:
df.dtypes.tolist()

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int6

In [86]:
df

,Estimated owners,Peak CCU,Required age,Price,DLC count,Windows,Mac,Linux,Metacritic score,User score,Positive,Negative,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,English,French,Japanese,Swedish,Italian,German,Arabic,Finnish,Greek,Hebrew,Persian,Russian,Korean,Captions available,Co-op,Commentary available,Cross-Platform Multiplayer,Full controller support,In-App Purchases,Includes Source SDK,Includes level editor,LAN Co-op,LAN PvP,MMO,Mods,Mods (require HL2),Multi-player,Online Co-op,Online PvP,Partial Controller Support,PvP,Remote Play Together,Remote Play on Phone,Remote Play on TV,Remote Play on Tablet,Shared/Split Screen,Shared/Split Screen Co-op,Shared/Split Screen PvP,Single-player,Stats,Steam Achievements,Steam Cloud,Steam Leaderboards,Steam Trading Cards,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,VR Support,Valve Anti-Cheat enabled,360 Video,Accounting,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Documentary,Early Access,Education,Episodic,Free to Play,Game Development,Gore,Indie,Massively Multiplayer,Movie,Nudity,Photo Editing,RPG,Racing,Sexual Content,Short,Simulation,Software Training,Sports,Strategy,Tutorial,Utilities,Video Production,Violent,Web Publishing
0,0,0,0,19.99,0,1,0,0,0,0,6,11,30,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0.99,0,1,1,0,0,0,53,5,12,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,4.99,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,5.99,0,1,1,1,0,0,3,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0.00,0,1,1,0,0,0,50,8,17,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63205,0,0,0,0.00,2,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
63206,0,3,0,9.99,0,1,0,0,0,0,2,0,15,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
63207,0,0,0,7.19,0,1,0,0,0,0,0,0,9,911,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
63208,0,0,0,2.99,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0


In [87]:
names = None
items = df

x = items.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
x_scaled = x_scaled / norm(x_scaled, axis=1)[:, None]
items = pd.DataFrame(x_scaled, columns=items.columns)

In [88]:
items

,Estimated owners,Peak CCU,Required age,Price,DLC count,Windows,Mac,Linux,Metacritic score,User score,Positive,Negative,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,English,French,Japanese,Swedish,Italian,German,Arabic,Finnish,Greek,Hebrew,Persian,Russian,Korean,Captions available,Co-op,Commentary available,Cross-Platform Multiplayer,Full controller support,In-App Purchases,Includes Source SDK,Includes level editor,LAN Co-op,LAN PvP,MMO,Mods,Mods (require HL2),Multi-player,Online Co-op,Online PvP,Partial Controller Support,PvP,Remote Play Together,Remote Play on Phone,Remote Play on TV,Remote Play on Tablet,Shared/Split Screen,Shared/Split Screen Co-op,Shared/Split Screen PvP,Single-player,Stats,Steam Achievements,Steam Cloud,Steam Leaderboards,Steam Trading Cards,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,VR Support,Valve Anti-Cheat enabled,360 Video,Accounting,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Documentary,Early Access,Education,Episodic,Free to Play,Game Development,Gore,Indie,Massively Multiplayer,Movie,Nudity,Photo Editing,RPG,Racing,Sexual Content,Short,Simulation,Software Training,Sports,Strategy,Tutorial,Utilities,Video Production,Violent,Web Publishing
0,0.0,0.000000,0.0,0.006670,0.000000,0.333326,0.000000,0.000000,0.0,0.0,3.469481e-07,0.000004,0.001018,0.000000,0.0,0.0,0.0,0.0,0.333326,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333326,0.0,0.000000,0.333326,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.333326,0.0,0.333326,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.333326,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.333326,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.333326,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.000240,0.000000,0.242536,0.242536,0.000000,0.0,0.0,2.229953e-06,0.000001,0.000296,0.000000,0.0,0.0,0.0,0.0,0.242536,0.242536,0.242536,0.0,0.242536,0.242536,0.0,0.0,0.0,0.0,0.0,0.242536,0.000000,0.0,0.0,0.0,0.0,0.242536,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.242536,0.242536,0.242536,0.0,0.0,0.0,0.242536,0.0,0.242536,0.0,0.242536,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.242536,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.242536,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.001888,0.000000,0.377964,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.377964,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.377964,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.377964,0.377964,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.377964,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.377964,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.001548,0.000000,0.258199,0.258199,0.258199,0.0,0.0,1.343753e-07,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.258199,0.258199,0.258199,0.0,0.258199,0.258199,0.0,0.0,0.0,0.0,0.0,0.258199,0.258199,0.0,0.0,0.0,0.0,0.258199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.258199,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.258199,0.0,0.0,0.258199,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.258199,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.000000,0.000000,0.377964,0.377964,0.000000,0.0,0.0,3.278425e-06,0.000003,0.000654,0.000000,0.0,0.0,0.0,0.0,0.377964,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.

In [89]:
import os

with open(os.path.expanduser(datadir+fname+'.pkl'), 'wb') as fout:
    pickle.dump([items,names], fout)